- 這裡是一個通用的函式 log_confusion_matrix()，能幫你把混淆矩陣 以圖和指標兩種方式 log 到 MLflow：

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import tempfile
import mlflow
from sklearn.metrics import confusion_matrix, classification_report

def log_classification_results(y_true, y_pred, labels=None, prefix="CM"):
    """
    Log confusion matrix (as image + metrics) and classification report to MLflow.

    Parameters:
    - y_true: true labels
    - y_pred: predicted labels
    - labels: label names (e.g., ['No', 'Yes']) for the confusion matrix
    - prefix: prefix for metrics (default: "CM")
    """
    # === 1. Confusion matrix ===
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    
    with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmpfile:
        plt.savefig(tmpfile.name)
        mlflow.log_artifact(tmpfile.name, artifact_path="plots")
    plt.close()
    
    # Log confusion matrix values
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            mlflow.log_metric(f"{prefix}_{i}_{j}", cm[i][j])

    # === 2. Classification report ===
    report = classification_report(y_true, y_pred, output_dict=True)
    
    for label, scores in report.items():
        if isinstance(scores, dict):
            for metric_name, value in scores.items():
                mlflow.log_metric(f"{prefix}_{label}_{metric_name}", value)
        else:
            # e.g., accuracy
            mlflow.log_metric(f"{prefix}_{label}", scores)


In [ ]:
y_pred_rf = rf_model.predict(X_test)

# 記錄 RF 模型的分類結果
log_classification_results(y_test, y_pred_rf, labels=[0, 1], prefix="RF")


- 📝 在 MLflow 中你會看到：
    - 類型	名稱範例
    - 📊 Metrics	RF_accuracy, RF_0_precision, RF_1_recall
    - 📈 圖像	plots/confusion_matrix.png